In [1]:
# coding=utf-8
from scapy.all import *
import binascii
import struct
import time

import crcmod
import libnum

from scapy.layers.inet import IP, UDP
from scapy.layers.l2 import Ether

import numpy as np

plc_cube = []
hmi_cube = []
#seq = [[],[]]

### Convert ip address string to integer
def ip2long(ip):
    """
    Convert an IP string to long
    """
    packedIP = socket.inet_aton(ip)
    return struct.unpack("!L", packedIP)[0]

def packet_analysis(pkt):
    if IP in pkt and TCP in pkt:
        #if len(pkt[TCP].payload) > 6:
        #print(pkt[CIP].SERVICE_CODES[pkt[CIP].service])
        #pkt.show()
        if(pkt[TCP].dport == 502):
            d_time = float(str(pkt.time))
            s_ip = ip2long(pkt[IP].src)
            d_ip = ip2long(pkt[IP].dst)
            #seq[0].append(pkt[TCP].seq)
            hmi_cube.append([s_ip,d_ip,d_time,-1])
        if(pkt[TCP].sport == 502):
            d_time = float(str(pkt.time))
            s_ip = ip2long(pkt[IP].src)
            d_ip = ip2long(pkt[IP].dst)
            #seq.append(pkt[TCP].seq)
            plc_cube.append([s_ip,d_ip,d_time,-1])


In [2]:
try:
    sniff(offline='', prn=packet_analysis, store=0, filter="host 172.27.224.250")
    #sniff(offline='DataSet/Dex2019_0002_attack3.pcap', prn=packet_analysis, store=0, count=100000, filter="src host 192.168.1.10 and dst host 192.168.1.20")
    plc_cube = np.asarray(plc_cube)
    hmi_cube = np.asarray(hmi_cube)
    print('Completed')

except Exception as sniff_dec_err:
    traceback.print_exc()
    print(str(sniff_dec_err))

libpcap is not available. Cannot compile filter !


Traceback (most recent call last):
  File "/Users/AB/opt/anaconda3/lib/python3.7/site-packages/scapy/arch/common.py", line 77, in compile_filter
    from scapy.libs.winpcapy import (
  File "/Users/AB/opt/anaconda3/lib/python3.7/site-packages/scapy/libs/winpcapy.py", line 38, in <module>
    raise OSError("Cannot find libpcap.so library")
OSError: Cannot find libpcap.so library

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-2-52cf8790bbc1>", line 2, in <module>
    sniff(offline='DataSet/captures1_v2/captures1_v2/clean/eth2dump-clean-1h_1.pcap', prn=packet_analysis, store=0, filter="host 172.27.224.250")
  File "/Users/AB/opt/anaconda3/lib/python3.7/site-packages/scapy/sendrecv.py", line 1036, in sniff
    sniffer._run(*args, **kwargs)
  File "/Users/AB/opt/anaconda3/lib/python3.7/site-packages/scapy/sendrecv.py", line 889, in _run
    )] = offline
  File "/Users/AB/opt/anaconda3/lib/python3.7/site-package

In [ ]:
print(hmi_cube.shape)
print(plc_cube.shape)
print(plc_cube[2][0])

In [ ]:
#print(CIP.SERVICE_CODES)
#hmi_data = hmi_cube[hmi_cube[:,3]==[76]]
hmi_data = hmi_cube[hmi_cube[:,0]==[ip2long('172.27.224.70')]]
hmi_data = hmi_data[hmi_data[:,1]==[ip2long('172.27.224.250')]]
hmi_data = hmi_data[:,2]
print(hmi_data.shape)
print("Sliding Windows Timespan :",max(hmi_data)-min(hmi_data))

In [ ]:
hmi_diff = np.diff(hmi_data)
print(hmi_diff.shape)

In [ ]:
from numpy import array, linspace
from sklearn.neighbors import KernelDensity
from matplotlib.pyplot import plot
from scipy.signal import argrelextrema
from scipy.signal import find_peaks, chirp, find_peaks, peak_widths

maxrange = 0.15
N = 1000
np.random.seed(1)
X = hmi_diff[:1000].reshape(-1,1)
X_plot = np.linspace(-0.0, maxrange, N)[:, np.newaxis]
hist, bin_edges = np.histogram(hmi_diff[:3000], bins=N, range=(0.0,maxrange), density=True)
bin_edges = bin_edges[:-1]
true_dens = hist[:]*maxrange*(1/N)
print(np.sum(true_dens))

fig, ax = plt.subplots(figsize=(16,9))
ax.plot(X_plot[:, 0], true_dens,label='Read TAG distribution')

kde = KernelDensity(kernel='gaussian', bandwidth=0.0005).fit(X)
log_dens = kde.score_samples(X_plot)
#print(kde.get_params())
dens = np.exp(log_dens)/1000

peaks, _ = find_peaks(dens)

valleys = peak_widths(dens, peaks, rel_height=0.8)

print(dens[peaks])
print(valleys[1])
print(valleys)
print("Xplot_peak",X_plot[peaks])
ax.plot(X_plot[:, 0], dens, color='darkgreen', lw=2,
        linestyle='-', label='normal operation distribution (KDE Gaussian)')
ax.plot(X_plot[peaks],dens[peaks], "gx")
#ax.plot(X_plot[valleys],dens[valleys], "go")
ax.plot(valleys[2]*maxrange*(1/N),valleys[1], "ro")
ax.plot(valleys[3]*maxrange*(1/N),valleys[1], "ro")

#stack position and values

nppeaks = np.hstack((X_plot[peaks],dens[peaks].reshape(-1,1)))
npbounds = np.stack((valleys[2]*maxrange*(1/N),valleys[3]*maxrange*(1/N),valleys[1]), axis=-1)

#print("normal peaks",nppeaks)
#print("normal bounds",npbounds)
#[position,amplitude,left,right,height]
npdata = np.concatenate((nppeaks,npbounds),axis=1)
np.save("np_bounds_CRITIS_dst.npy",npdata)

ax.title.set_text("PLC-to-HMI arrival plots")
ax.legend(loc='upper right')
ax.plot(X[:, 0], -0.005 - 0.01 * np.random.random(X.shape[0]), '+k')

ax.set_xlim(-0.0, maxrange)
ax.set_ylim(-0.02, 0.3)

s = X_plot[:, 0]
e = np.exp(log_dens)/1000
a = X
mi, ma = argrelextrema(e, np.less)[0], argrelextrema(e, np.greater)[0]
print("Minima:", s[mi])
print("Maxima:", s[ma])